In [1]:
import psycopg2 as ps
from env_vars import *

In [3]:
# create connection and cursor    
conn = ps.connect(host=POSTGRES_ADDRESS,
                  database=POSTGRES_DBNAME,
                  user=POSTGRES_USERNAME,
                  password=POSTGRES_PASSWORD,
                  port=POSTGRES_PORT)
cur = conn.cursor()